In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [2]:
import numpy as np

from lint_analysis.bin_counts.models import BinCount

In [3]:
series = BinCount.token_series('random')

In [4]:
from joblib import Memory

mem = Memory('cache')

token_counts = mem.cache(BinCount.token_counts)

tokens = token_counts(10000)

In [ ]:
variances = []

for i, (token, count) in enumerate(tokens.items()):
    
    series = BinCount.token_series(token)
    
    variances.append(dict(
        token=token,
        count=count,
        variance=series.var()
    ))
    
    if i % 1000 == 0:
        print(i)

0


In [ ]:
import pandas as pd